In [1]:
import sys
import os, glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from xeofs.xarray import EOF
import gc
import collections
import scipy
from sklearn.metrics import mean_squared_error

In [5]:
# Define project repo path
inDirName = '/Users/smurugan9/research/aislens/aislens_emulation/'

# DATA FILE PATHS

# Data containing regridded flux and SSH for 150 years
regriddedFluxSSH_filepath = 'data/interim/RegriddedFluxSSH.nc'

# File contains all defined ice shelves
iceShelvesShape_filepath = 'data/interim/iceShelves.geojson'

# Folder for output figures
figures_folderpath = 'reports/figures/' 

interim_data_folder = 'data/interim/'
rmse_results_folder = 'rmse_grid_220911/'
flux_dedrafted_data_path = 'dedrafted_flux_IMBIE/'
randomized_realizations_path = 'randomized_realizations/'
flux_dedrafted_iceshelves_data_path = 'iceshelves_dedrafted_flux/'
reconstructions_neofs_path = 'reconstructions_neofs/'
inFilesDir = inDirName+interim_data_folder+rmse_results_folder

In [6]:
flux_clean = xr.open_dataset(inDirName+interim_data_folder+'flux_clean')
flux_clean = flux_clean.timeMonthly_avg_landIceFreshwaterFlux
# Drop all NaN-valued grid points
flux_clean = flux_clean.dropna('time',how='all')
flux_clean = flux_clean.dropna('y',how='all')
flux_clean = flux_clean.dropna('x',how='all')
flux_clean = flux_clean.drop("month")

In [7]:
%%time
model = EOF(flux_clean)
model.solve()
xeofs_eofs = model.eofs()
xeofs_pcs = model.pcs()
xeofs_n_modes = model.n_modes

CPU times: user 2min 29s, sys: 51.8 s, total: 3min 21s
Wall time: 1min 3s


In [9]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 1
t_length = xeofs_pcs.shape[0]

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,xeofs_pcs.shape[0],xeofs_pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(xeofs_n_modes):
        fl = xeofs_pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_fourier = np.fft.rfft(fl)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
        print('calculated ifft for realization {}, mode: {}'.format(i,m))

calculated ifft for realization 0, mode: 0
calculated ifft for realization 0, mode: 1
calculated ifft for realization 0, mode: 2
calculated ifft for realization 0, mode: 3
calculated ifft for realization 0, mode: 4
calculated ifft for realization 0, mode: 5
calculated ifft for realization 0, mode: 6
calculated ifft for realization 0, mode: 7
calculated ifft for realization 0, mode: 8
calculated ifft for realization 0, mode: 9
calculated ifft for realization 0, mode: 10
calculated ifft for realization 0, mode: 11
calculated ifft for realization 0, mode: 12
calculated ifft for realization 0, mode: 13
calculated ifft for realization 0, mode: 14
calculated ifft for realization 0, mode: 15
calculated ifft for realization 0, mode: 16
calculated ifft for realization 0, mode: 17
calculated ifft for realization 0, mode: 18
calculated ifft for realization 0, mode: 19
calculated ifft for realization 0, mode: 20
calculated ifft for realization 0, mode: 21
calculated ifft for realization 0, mode: 2

calculated ifft for realization 0, mode: 450
calculated ifft for realization 0, mode: 451
calculated ifft for realization 0, mode: 452
calculated ifft for realization 0, mode: 453
calculated ifft for realization 0, mode: 454
calculated ifft for realization 0, mode: 455
calculated ifft for realization 0, mode: 456
calculated ifft for realization 0, mode: 457
calculated ifft for realization 0, mode: 458
calculated ifft for realization 0, mode: 459
calculated ifft for realization 0, mode: 460
calculated ifft for realization 0, mode: 461
calculated ifft for realization 0, mode: 462
calculated ifft for realization 0, mode: 463
calculated ifft for realization 0, mode: 464
calculated ifft for realization 0, mode: 465
calculated ifft for realization 0, mode: 466
calculated ifft for realization 0, mode: 467
calculated ifft for realization 0, mode: 468
calculated ifft for realization 0, mode: 469
calculated ifft for realization 0, mode: 470
calculated ifft for realization 0, mode: 471
calculated

calculated ifft for realization 0, mode: 893
calculated ifft for realization 0, mode: 894
calculated ifft for realization 0, mode: 895
calculated ifft for realization 0, mode: 896
calculated ifft for realization 0, mode: 897
calculated ifft for realization 0, mode: 898
calculated ifft for realization 0, mode: 899
calculated ifft for realization 0, mode: 900
calculated ifft for realization 0, mode: 901
calculated ifft for realization 0, mode: 902
calculated ifft for realization 0, mode: 903
calculated ifft for realization 0, mode: 904
calculated ifft for realization 0, mode: 905
calculated ifft for realization 0, mode: 906
calculated ifft for realization 0, mode: 907
calculated ifft for realization 0, mode: 908
calculated ifft for realization 0, mode: 909
calculated ifft for realization 0, mode: 910
calculated ifft for realization 0, mode: 911
calculated ifft for realization 0, mode: 912
calculated ifft for realization 0, mode: 913
calculated ifft for realization 0, mode: 914
calculated

calculated ifft for realization 0, mode: 1352
calculated ifft for realization 0, mode: 1353
calculated ifft for realization 0, mode: 1354
calculated ifft for realization 0, mode: 1355
calculated ifft for realization 0, mode: 1356
calculated ifft for realization 0, mode: 1357
calculated ifft for realization 0, mode: 1358
calculated ifft for realization 0, mode: 1359
calculated ifft for realization 0, mode: 1360
calculated ifft for realization 0, mode: 1361
calculated ifft for realization 0, mode: 1362
calculated ifft for realization 0, mode: 1363
calculated ifft for realization 0, mode: 1364
calculated ifft for realization 0, mode: 1365
calculated ifft for realization 0, mode: 1366
calculated ifft for realization 0, mode: 1367
calculated ifft for realization 0, mode: 1368
calculated ifft for realization 0, mode: 1369
calculated ifft for realization 0, mode: 1370
calculated ifft for realization 0, mode: 1371
calculated ifft for realization 0, mode: 1372
calculated ifft for realization 0,

In [73]:
ts = np.random.rand(3,3)
ts

array([[0.07610405, 0.11837383, 0.61410457],
       [0.75989172, 0.63143107, 0.87207218],
       [0.8690375 , 0.93360079, 0.78549462]])

In [85]:
def phase_rand(data, n):
    t, m = data.shape
    fl = data[:,0:-1]
    return fl

In [86]:
tmpfl = phase_rand(ts,2)
tmpfl.shape

(3, 1)

In [88]:
fl_fourier = np.fft.rfft(ts)
fl_fourier

array([[ 0.80858245+0.j        , -0.29013515+0.42931541j],
       [ 2.26339496+0.j        ,  0.00814009+0.20840131j],
       [ 2.58813291+0.j        ,  0.0094898 -0.12826371j]])

In [45]:
fl_fourier = np.fft.rfft(fl)
random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
fl_fourier_new = fl_fourier*random_phases
new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
return new_fl

array([[0.97495375, 0.33781873],
       [0.61530182, 0.94900524],
       [0.96285879, 0.99152613]])

In [22]:
pressure = np.array([1000, 850, 500, 300])
temps = np.linspace(20, 30, 24).reshape(4, 3, 2)

temps * np.exp(pressure[:, np.newaxis, np.newaxis] / 1000)

array([[[54.36563657, 55.54749823],
        [56.7293599 , 57.91122156],
        [59.09308323, 60.27494489]],

       [[52.89636361, 53.91360137],
        [54.93083913, 55.94807689],
        [56.96531466, 57.98255242]],

       [[41.57644944, 42.29328477],
        [43.01012011, 43.72695544],
        [44.44379078, 45.16062611]],

       [[37.56128856, 38.14818369],
        [38.73507883, 39.32197396],
        [39.90886909, 40.49576423]]])

In [ ]:
def vectorized_app2(volume, roi, radius):
    m,n,r = volume.shape
    vals = ((np.arange(m)-roi[0])**2)[:,None,None] + \
           ((np.arange(n)-roi[1])**2)[:,None] + ((np.arange(r)-roi[2])**2)
    return vals < radius**2